In [ ]:
!pip install textattack[tensorflow,optional]

In [ ]:
!pip install tensorflow==2.14

  Using cached tensorflow-2.14.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (489.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 6.9 MB/s eta 0:00:00
  Using cached tensorboard-2.14.1-py3-none-any.whl (5.5 MB)
  Using cached tensorflow_estimator-2.14.0-py2.py3-none-any.whl (440 kB)
  Using cached keras-2.14.0-py3-none-any.whl (1.7 MB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl (18 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 70.9 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully uninstalled flatbuffers-1.12
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-ser

In [ ]:
!textattack peek-dataset --dataset-from-huggingface glue 'qnli'

/bin/bash: line 1: textattack: command not found


In [ ]:
import numpy as np
import tensorflow as tf

import torch
import textattack
import transformers

from textattack import AttackArgs
from textattack.datasets import Dataset
from textattack import Attacker

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


In [ ]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/albert-base-v2-rotten-tomatoes")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/albert-base-v2-rotten-tomatoes")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)


config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/46.7M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [ ]:
dataset = textattack.datasets.HuggingFaceDataset("rotten_tomatoes", split="train")

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

textattack: Loading datasets dataset rotten_tomatoes, split train.


In [ ]:
train_dataset = textattack.datasets.HuggingFaceDataset("rotten_tomatoes", split="train")
eval_dataset = textattack.datasets.HuggingFaceDataset("rotten_tomatoes", split="test")

textattack: Loading datasets dataset rotten_tomatoes, split train.
textattack: Loading datasets dataset rotten_tomatoes, split test.


In [ ]:
attack = textattack.attack_recipes.CLARE2020.build(model_wrapper)


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
attack_args = textattack.AttackArgs(
    num_examples=100,
    checkpoint_interval=5,
    checkpoint_dir="checkpoints"
)

In [ ]:
attacker = textattack.Attacker(attack, dataset, attack_args)

results = attacker.attack_dataset()
ppl = textattack.metrics.quality_metrics.Perplexity().calculate(results)
usem = textattack.metrics.quality_metrics.USEMetric().calculate(results)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapMaskedLM(
        (method):  bae
        (masked_lm_name):  RobertaForCausalLM
        (max_length):  512
        (max_candidates):  50
        (min_confidence):  0.0005
      )
    (1): WordInsertionMaskedLM(
        (masked_lm_name):  RobertaForCausalLM
        (max_length):  512
        (max_candidates):  50
        (min_confidence):  0.0
      )
    (2): WordMergeMaskedLM(
        (masked_lm_name):  RobertaForCausalLM
        (max_length):  512
        (max_candidates):  50
        (min_confidence):  0.005
      )
    )
  (constraints): 
    (0): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.7
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 




  1%|          | 1/100 [01:24<2:19:50, 84.75s/it]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/100 [01:24<2:19:51, 84.76s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (92%)]] --> [[Negative (94%)]]

the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .

the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even [[barely]] greater than arnold schwarzenegger , jean-claud van damme or steven segal .





[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   2%|▏         | 2/100 [03:06<2:31:55, 93.01s/it]
[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:   2%|▏         | 2/100 [03:06<2:31:55, 93.02s/it]
[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:   3%|▎         | 3/100 [03:06<1:40:16, 62.03s/it]

--------------------------------------------- Result 2 ---------------------------------------------
[[Positive (98%)]] --> [[Negative (79%)]]

the [[gorgeously]] elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth .

the [[overly]] elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth .


--------------------------------------------- Result 3 ---------------------------------------------
[[Negative (75%)]] --> [[[SKIPPED]]]

effective but too-tepid biopic




...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2171 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
print(ppl)

{'avg_original_perplexity': 71.4, 'avg_attack_perplexity': 88.35}


In [ ]:
print(usem)

{'avg_attack_use_score': 0.9}


In [ ]:
training_args = textattack.TrainingArgs(
    num_epochs=3,
    num_clean_epochs=1,
    num_train_adv_examples=100,
    learning_rate=5e-5,
    per_device_train_batch_size=10,
    gradient_accumulation_steps=4,
    query_budget_train=120,
    save_last=True,
    load_best_model_at_end=True
)

trainer = textattack.Trainer(
    model_wrapper,
    "classification",
    attack,
    train_dataset,
    eval_dataset,
    training_args
)
m=trainer.train()


textattack: Writing logs to ./outputs/2023-12-15-20-49-30-197990/train_log.txt.
textattack: Wrote original training args to ./outputs/2023-12-15-20-49-30-197990/training_args.json.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
textattack: ***** Running training *****
textattack:   Num examples = 8530
textattack:   Num epochs = 3
textattack:   Num clean epochs = 1
textattack:   Instantaneous batch size per device = 10
textattack:   Total train batch size (w. parallel, distributed & accumulation) = 40
textattack:   Gradient accumulation steps = 4
textattack:   Total optimization steps = 646
textattack: ==========================================================
textattack: Epoch 1
textattack: Running clean epoch 1/1
Loss 0.17679: 

Loss 0.20297: 100%|██████████| 863/863 [13:16<00:00,  1.08it/s]
textattack: Train accuracy: 91.49%
textattack: Eval accuracy: 79.55%
textattack: ==========================================================
textattack: Epoch 3
textattack: Attacking model to generate new adversarial training set...

  0%|          | 0/100 [00:00<?, ?it/s]
[Succeeded / Failed / Skipped / Total] 0 / 1 / 0 / 1:   0%|          | 0/100 [00:10<?, ?it/s]
[Succeeded / Failed / Skipped / Total] 0 / 2 / 0 / 2:   0%|          | 0/100 [00:23<?, ?it/s]
[Succeeded / Failed / Skipped / Total] 0 / 2 / 0 / 2:   1%|          | 1/100 [00:30<49:54, 30.25s/it]
[Succeeded / Failed / Skipped / Total] 1 / 2 / 0 / 3:   1%|          | 1/100 [00:30<49:56, 30.27s/it]
[Succeeded / Failed / Skipped / Total] 1 / 2 / 0 / 3:   2%|▏         | 2/100 [00:42<34:25, 21.07s/it]
[Succeeded / Failed / Skipped / Total] 2 / 2 / 0 / 4:   2%|▏         | 2/100 [00:42<34:26, 21.08s/it]
[Succeeded / Failed / Skipped / Total] 2 / 3 / 0 / 5:   2%|▏       

Loss 0.20169: 100%|██████████| 863/863 [13:15<00:00,  1.09it/s]
textattack: Train accuracy: 92.64%
textattack: Eval accuracy: 83.58%
textattack: Wrote README to ./outputs/2023-12-15-20-49-30-197990/README.md.


In [ ]:
# import the CheckListAugmenter
from textattack.augmentation import CLAREAugmenter

# Alter default values if desired
augmenter = CLAREAugmenter(pct_words_to_swap=0.2, transformations_per_example=5)
s = "I'd love to go to Japan but the tickets are 500 dollars"
# Augment
augmenter.augment(s)

If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`
[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [4:11:32<37:43:49, 15092.13s/it]


["I'd kinda love to go to Japan but the tickets are 500 dollars",
 "I'd love to go to Japan but the exact tickets are 500 dollars",
 "I'd love to go to Japan but the tourist tickets are 500 dollars",
 "I'd prefer to go to Japan but mainly the tickets are 500 dollars",
 "I'd really love to go to Australia but the tickets are 500 dollars"]

In [ ]:
attackedModel = transformers.AutoModelForSequenceClassification.from_pretrained("./outputs/2023-12-15-20-49-30-197990/best_model/")
attackedTokenizer = transformers.AutoTokenizer.from_pretrained("./outputs/2023-12-15-20-49-30-197990/best_model/")
attacked_model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

In [ ]:
attackTrainedModel = textattack.attack_recipes.CLARE2020.build(attacked_model_wrapper)


If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`
textattack: Unknown if model of class <class 'transformers.models.albert.modeling_albert.AlbertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [ ]:
attack_args = textattack.AttackArgs(
    num_examples=100,
    checkpoint_interval=5,
    checkpoint_dir="checkpoints"
)

In [ ]:
attackerOnTrained = textattack.Attacker(attackTrainedModel, dataset, attack_args)

trainedResults = attackerOnTrained.attack_dataset()
trainedPPL = textattack.metrics.quality_metrics.Perplexity().calculate(results)
usemTrained = textattack.metrics.quality_metrics.USEMetric().calculate(results)

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapMaskedLM(
        (method):  bae
        (masked_lm_name):  RobertaForCausalLM
        (max_length):  512
        (max_candidates):  50
        (min_confidence):  0.0005
      )
    (1): WordInsertionMaskedLM(
        (masked_lm_name):  RobertaForCausalLM
        (max_length):  512
        (max_candidates):  50
        (min_confidence):  0.0
      )
    (2): WordMergeMaskedLM(
        (masked_lm_name):  RobertaForCausalLM
        (max_length):  512
        (max_candidates):  50
        (min_confidence):  0.005
      )
    )
  (constraints): 
    (0): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.7
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:   1%|          | 1/100 [01:27<2:23:47, 87.15s/it]

--------------------------------------------- Result 1 ---------------------------------------------
[[Positive (90%)]] --> [[Negative (89%)]]

the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even [[greater]] than arnold schwarzenegger , jean-claud van damme or steven segal .

the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even [[worse]] than arnold schwarzenegger , jean-claud van damme or steven segal .




[Succeeded / Failed / Skipped / Total] 2 / 0 / 0 / 2:   2%|▏         | 2/100 [03:06<2:32:34, 93.42s/it]

--------------------------------------------- Result 2 ---------------------------------------------
[[Positive (92%)]] --> [[Negative (69%)]]

the [[gorgeously]] elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth .

the [[overly]] elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth .




...

[Succeeded / Failed / Skipped / Total] 94 / 0 / 6 / 100: 100%|██████████| 100/100 [1:49:14<00:00, 65.55s/it]textattack: Saving checkpoint under "checkpoints/1702690179494.ta.chkpt" at 2023-12-16 01:29:39 after 100 attacks.


--------------------------------------------- Result 99 ---------------------------------------------
[[Positive (94%)]] --> [[Negative (71%)]]

. . . a roller-coaster ride of a movie

. . . a roller-coaster ride of a [[bad]] movie


--------------------------------------------- Result 100 ---------------------------------------------
[[Negative (92%)]] --> [[[SKIPPED]]]

i enjoyed time of favor while i was watching it , but i was surprised at how quickly it faded from my memory .







[Succeeded / Failed / Skipped / Total] 94 / 0 / 6 / 100: 100%|██████████| 100/100 [1:49:14<00:00, 65.55s/it]


+-------------------------------+---------+
| Attack Results                |         |
+-------------------------------+---------+
| Number of successful attacks: | 94      |
| Number of failed attacks:     | 0       |
| Number of skipped attacks:    | 6       |
| Original accuracy:            | 94.0%   |
| Accuracy under attack:        | 0.0%    |
| Attack success rate:          | 100.0%  |
| Average perturbed word %:     | 30.98%  |
| Average num. words per input: | 18.4    |
| Avg num queries:              | 1695.05 |
+-------------------------------+---------+

Token indices sequence length is longer than the specified maximum sequence length for this model (2171 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
print(trainedPPL)

{'avg_original_perplexity': 71.4, 'avg_attack_perplexity': 88.35}


In [ ]:
print(usemTrained)

{'avg_attack_use_score': 0.9}
